In [5]:
import pandas as pd
from pathlib import Path

In [18]:
DATA_DIR = Path().absolute().parent / "data" / "ngs_stratified"
METADATA_DIR = DATA_DIR / "metadata"

In [21]:
read_run_df = pd.read_csv(METADATA_DIR / "read_run.csv")
sample_df = pd.read_csv(METADATA_DIR / "sample.csv")
study_df = pd.read_csv(METADATA_DIR / "study.csv").set_index("study_accession")
ngs_sample_df = pd.read_csv(DATA_DIR / "ngs_sample_clean.csv")

In [47]:
ngs_sample_df["run_accession"] = ngs_sample_df["sequence_id"].str.split(".").str[0]
metadata_df = read_run_df[["sample_accession", "study_accession", "run_accession", "library_source"]].merge(
    sample_df[["sample_accession", "scientific_name"]], on="sample_accession"
)
filtered_metadata_df = metadata_df[
    (metadata_df["library_source"] == "TRANSCRIPTOMIC") & (metadata_df["scientific_name"] == "Homo sapiens")
]
filtered_ngs_sample_df = ngs_sample_df.merge(filtered_metadata_df.drop(columns=["study_accession"]), on="run_accession")

filtered_ngs_sample_df = filtered_ngs_sample_df.drop_duplicates(subset=["sequence"]) 
filtered_ngs_sample_df.to_csv(DATA_DIR / "ngs_sample_human.csv", index=False)
with open(DATA_DIR / "ngs_sample_human_nt.fasta", "w") as file:
    for record in filtered_ngs_sample_df[["sequence_id", "sequence"]].itertuples():
        file.write(f">{record.sequence_id}\n")
        file.write(f"{record.sequence}\n")